In [1]:

# # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# # TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# # THEN FEEL FREE TO DELETE THIS CELL.
# # NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# # ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# # NOTEBOOK.

# import os
# import sys
# from tempfile import NamedTemporaryFile
# from urllib.request import urlopen
# from urllib.parse import unquote, urlparse
# from urllib.error import HTTPError
# from zipfile import ZipFile
# import tarfile
# import shutil

# CHUNK_SIZE = 40960
# DATA_SOURCE_MAPPING = 'enodparallel:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5117966%2F8562046%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240602%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240602T101559Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5fe51b23945cf5e2d66a92dede6be085eca04c7378bd1e50933a4d2abc9ffcfeda6feda569e8439edc65c2e2f4824b6861a1d4c9d709717ab1725ba9a9a913f160b857f244effd4bfa420c87c59dc7849c065b5f010560eea8f6d3dc73b21c6a7a422c41706c5953b083cb3e966a66f34f97c6f53b5d539328f833ab2e075c519a41181051ff3393cf16b15f4ac8c3b4432031b7c0cb70797ff0e6ec2d5c836f214eaed81f0e46e4ffec4edee27ca0e40d8c6f86ceefcc6a4336b050a5a14833613b2e944b166ee4a74460e77966b1cbd2abf2a72245dddbfb6d575d0b4dfd5cfb5f6ae6054eaf25c2a866eea1bf5b3273b30ff0aa6b54340d950698d5a9dc49'

# KAGGLE_INPUT_PATH='/kaggle/input'
# KAGGLE_WORKING_PATH='/kaggle/working'
# KAGGLE_SYMLINK='kaggle'

# !umount /kaggle/input/ 2> /dev/null
# shutil.rmtree('/kaggle/input', ignore_errors=True)
# os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
# os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

# try:
#   os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
# except FileExistsError:
#   pass
# try:
#   os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
# except FileExistsError:
#   pass

# for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
#     directory, download_url_encoded = data_source_mapping.split(':')
#     download_url = unquote(download_url_encoded)
#     filename = urlparse(download_url).path
#     destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
#     try:
#         with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
#             total_length = fileres.headers['content-length']
#             print(f'Downloading {directory}, {total_length} bytes compressed')
#             dl = 0
#             data = fileres.read(CHUNK_SIZE)
#             while len(data) > 0:
#                 dl += len(data)
#                 tfile.write(data)
#                 done = int(50 * dl / int(total_length))
#                 sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
#                 sys.stdout.flush()
#                 data = fileres.read(CHUNK_SIZE)
#             if filename.endswith('.zip'):
#               with ZipFile(tfile) as zfile:
#                 zfile.extractall(destination_path)
#             else:
#               with tarfile.open(tfile.name) as tarfile:
#                 tarfile.extractall(destination_path)
#             print(f'\nDownloaded and uncompressed: {directory}')
#     except HTTPError as e:
#         print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
#         continue
#     except OSError as e:
#         print(f'Failed to load {download_url} to path {destination_path}')
#         continue

# print('Data source import complete.')


In [2]:
import numpy as np
import pandas as pd

In [3]:
!pip install transformers datasets torch sacrebleu peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.

### Preprocessing Datasets

In [4]:
# import os
# def load_dataset_from_txt(file_path):
#     with open(file_path, 'r', encoding='utf-8') as file:
#         data = file.read().splitlines()  # splitlines() handles different newline characters
#     return data

In [5]:
# dir_path = '/kaggle/input/enodparallel'
# train_path= dir_path+'/train.txt'
# test_path = dir_path+'/test.txt'
# validate_path = dir_path+'/validate.txt'

In [6]:
# train_data = load_dataset_from_txt(train_path)
# test_data = load_dataset_from_txt(test_path)
# validate_data = load_dataset_from_txt(validate_path)

In [7]:
# def preprocess_dataset(dataset):
#     processed_data = []
#     for item in dataset:
#         if '||' in item:
#             en, od = item.split('||', 1)  # Split only on the first occurrence of '||'
#             processed_data.append({'en': en.strip(), 'od': od.strip()})
#     return processed_data

In [8]:
# processed_train = preprocess_dataset(train_data)
# processed_test = preprocess_dataset(test_data)
# processed_val = preprocess_dataset(validate_data)

In [9]:
# from datasets import Dataset, DatasetDict
# df_train = Dataset.from_list(processed_train)
# df_test = Dataset.from_list(processed_test)
# df_val = Dataset.from_list(processed_val)

In [10]:
# df_train

In [11]:
# dataset_dict = DatasetDict({
#     'train': df_train,
#     'test':df_test,
#     'val':df_val
# })

In [12]:
# dataset_dict

In [13]:
# from transformers import T5ForConditionalGeneration, T5Tokenizer

# model_name = 'jbochi/madlad400-3b-mt'
# model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto")
# tokenizer = T5Tokenizer.from_pretrained(model_name)

In [14]:
#training for question answering
from datasets import load_dataset
dataset = load_dataset("ai4bharat/IndicQA", 'indicqa.or')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for ai4bharat/IndicQA contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ai4bharat/IndicQA
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warn

Generating test split: 0 examples [00:00, ? examples/s]

In [15]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1680
    })
})


In [16]:
# for example in dataset['test']:
#     sequence_values = example['question']
#     print(sequence_values)

In [17]:
# for example in dataset['test']:
#     sequence_values = example['answers']
#     print(sequence_values)

In [18]:
# for example in dataset['test']:
#     sequence_values = example['context']
#     print(sequence_values)

In [19]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")

def tokenize_function(examples):
    # Concatenate the context and question to form the input for the model
    inputs = [f"question: {q} context: {c}" for q, c in zip(examples['question'], examples['context'])]
    # Tokenize the inputs
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=128)

    # Extract the correct answer from the 'answers' field
    # Assuming the 'answers' field is a list of dictionaries with 'text' as the key for the answer text
    # and 'answer_start' as the key for the start index of the answer in the context
    # Here we are using the first answer in the list
    answers = [answer['text'][0] for answer in examples['answers']]

    # Tokenize the answers
    labels = tokenizer(text_target=answers, padding="max_length", truncation=True, max_length=128)

    # Set the labels to the tokenized answers
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Assuming `dataset_dict` is your DatasetDict with the 'test' split
tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

In [20]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")

# def tokenize_function(examples):
#     model_inputs = tokenizer(examples['en'], padding="max_length", truncation=True, max_length=128)
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples['od'], padding="max_length", truncation=True, max_length=128)
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs
# tokenized_datasets = dataset_dict.map(tokenize_function, batched = True)

In [21]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1680
    })
})

In [22]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import torch
from datasets import load_metric, load_dataset
from peft import get_peft_model, LoraConfig

In [23]:
lora_config = LoraConfig(
r = 4,
lora_alpha = 32,
lora_dropout = 0.1,
bias = "none"
)

model = get_peft_model(model, lora_config)

training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
    save_steps=500,
    logging_steps=100,
    load_best_model_at_end=True,
)



# Prepare data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Prepare metric for evaluation with trust_remote_code=True
metric = load_metric("sacrebleu", trust_remote_code=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    return result

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-23-942c9aebc565>:33: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu", trust_remote_code=True)


In [40]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1680
    })
})

In [25]:
# tokenized_datasets['test']

In [26]:
# len(train_dataset)

In [27]:
# len(val_dataset)

In [28]:
from sklearn.model_selection import train_test_split

# Split the 'test' dataset into train and validation sets
split_datasets = tokenized_datasets['test'].train_test_split(test_size=0.2)
train_dataset = split_datasets['train']
val_dataset = split_datasets['test']  # This is the validation set


In [38]:
# train_dataset[5:10]

{'id': ['216', '315', '872', '38', '121'],
 'context': ['କକ୍ଷ ମଝିରେ ଏକ-ଦୁଇ ଟନ ଓଜନର (ଶୌଣ୍ଡେଲିୟର) ଲଗା ହୋଇଛି , ଯାହା ୩୩ ମିଟର ଉଞ୍ଚା ଛାତରୁ ଓହଳି ରହିଛି । ଏହି ବିଶାଳ କକ୍ଷର ଚାରି କୋଣରେ ସ୍ଥିତ ରହିଛି ଗୋଟିଏ କକ୍ଷ ପ୍ରତି କୋଣ। ଏଥିରୁ ଦୁଇ ଷ୍ଟେଟ ଡ଼ାଇନିଂ କକ୍ଷ, ଗୋଟେ ଷ୍ଟେଟ ଉପର କକ୍ଷ ଏବଂ ଗୋଟେ ଷ୍ଟେଟ ପୁସ୍ତକାଳୟ । ଅନ୍ୟ କକ୍ଷ ଗଳି ଯେପରି ମଧ୍ୟ ରହିଛି ତାହାର ଚତୁର୍ପାଶ୍ୱ ଖୋଲା ରହିଛି । ଏକ ବୃହତ ଭୋଜନ କକ୍ଷ , ଏକ ବୈଠକ କକ୍ଷ, ବିଲିୟାର୍ଡ଼ କକ୍ଷ ଇତ୍ୟାଦି ରହିଛି । ପ୍ରାସାଦରେ ସର୍ବତ୍ର ବହୁ ସ୍ଥାନରେ ଜଳର ଫବାରେ ଏବଂ ବେସିନ ତିଆରି ହୋଇଛି ।',
  'କୋଭିଡ-୧୯ ସଂକ୍ରମଣରୁ ଭଲ ହେବା ପାଇଁ ଏଯାବତ କୌଣସି ଔଷଧ ବାହାରି ନାହିଁ । କେବଳ ସଙ୍ଗରୋଧ ଓ ସଚେତନତା ହିଁ ଏକମାତ୍ର ଉପାୟ [୮]। ପ୍ରାଥମିକ ପର୍ଯ୍ୟାୟରେ ଓଡ଼ିଶାରେ ଦୁଇଟି କୋଭିଡ-୧୯ ଘଟଣା ନିଶ୍ଚିତ ହୋଇଥିଲା । ମାର୍ଚ୍ଚ ୨୧ ତାରିଖରେ ପ୍ରଥମ ପର୍ଯ୍ୟ। ୟରେ ୫ଟି ଜିଲ୍ଲାରେ ଏବଂ ଦ୍ୱିତୀୟ ପର୍ଯ୍ୟ। ୟରେ ଆଉ ୯ଟି ଜିଲ୍ଲାରେ ଓଡ଼ିଶା ସରକାର ତାଲାବନ୍ଦ ଘୋଷଣା କରିଥିଲେ । ପରେ ରାଜ୍ୟର ସମସ୍ତ ଜିଲ୍ଲାରେ ୨୪ ତାରିଖ ମଧ୍ୟରାତ୍ରୀରୁ ୫ ଦିନ ପାଇଁ ଲକଡାଉନ ଘୋଷଣା କରିବା ଅବସରରେ କେନ୍ଦ୍ର ସରକାର ସାରା ଭାରତରେ ୨୪ ତାରିଖ ମଧ୍ୟ ରାତ୍ରିରୁ ଏପ୍ରିଲ ୧୫ ଯାଏଁ ତାଲାବନ୍ଦ ଘୋଷଣା କରିଥିଲେ । ଓଡ଼ିଶା କ୍ୟାବିନେଟ ବୈଠକରେ ୯ ଏପ୍ରିଲ ଦିନ ଲକଡାଉନ

In [30]:
# val_dataset[5:10]

In [31]:

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

Step,Training Loss,Validation Loss,Bleu
100,0.000000,nan,0.000000
200,0.000000,nan,0.000000
300,0.000000,nan,0.000000
400,0.000000,nan,0.000000
500,0.000000,nan,0.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=504, training_loss=0.0, metrics={'train_runtime': 285.7963, 'train_samples_per_second': 14.108, 'train_steps_per_second': 1.763, 'total_flos': 691605237399552.0, 'train_loss': 0.0, 'epoch': 3.0})

In [36]:
# Generate predictions using the trained model
predictions = trainer.predict(tokenized_datasets['test'])

# Decode the predictions
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [68]:
from utils import compute_score
from utils import f1_score
# Calculate the BLEU score
bleu_score = compute_score(predictions=decoded_preds, references=references)
f1 = f1_score(decoded_preds, references)
print(f"BLEU SCORE: {bleu_score}")
print(f"F1 score of the decoded preds: {f1}")

BLEU SCORE: 0.4050021510445334
F1 score of the decoded preds: 0.5322555704901326


In [35]:
# Print some examples
for i, pred in enumerate(decoded_preds[:5]):
    print(f"Example {i + 1}:")
    print(f"Prediction: {tokenized_datasets['test'][i]['question']}")
    print(f"Reference: {tokenized_datasets['test'][i]['answers']}")

Example 1:
Prediction: କେଉଁ ସରକାର ରିପୋର୍ଟ କରିଛନ୍ତି ଯେ ଆରଏସଏସ କୌଣସି ସରକାରୀ ଆଦେଶକୁ ଉଲ୍ଲଂଘନ କରି ନାହାନ୍ତି?
Reference: {'text': ['ବ୍ରିଟିଶ ସରକାର'], 'answer_start': [717]}
Example 2:
Prediction: କେଉଁ ବର୍ଷରେ ଆରଏସଏସ ନେତାମାନେ ସମସ୍ତ କାର୍ଯ୍ୟକଳାପରୁ ଦୂରେଇ ରହିଥିଲେ?
Reference: {'text': ['୧୯୪୨ ମସିହାରେ'], 'answer_start': [413]}
Example 3:
Prediction: ଆରଏସଏସର କେଉଁ ନେତା କହିଥିଲେ ଯେ ଆରଏସଏସ ଭାରତ ଛାଡିବା ଆନ୍ଦୋଳନକୁ ସମର୍ଥନ କରୁନାହିଁ?
Reference: {'text': ['ଏମ.ଏସ. ଗୋଲୱାଲକର'], 'answer_start': [169]}
Example 4:
Prediction: କାର୍ଯ୍ୟକଳାପରୁ ନିବୃତ୍ତ ହେବାକୁ ଆରଏସଏସ ନେତାମାନଙ୍କୁ କିଏ ନିର୍ଦ୍ଦେଶ ଦେଇଛନ୍ତି?
Reference: {'text': ['ଏମ.ଏସ. ଗୋଲୱାଲକର'], 'answer_start': [169]}
Example 5:
Prediction: ବ୍ରିଟିଶ ସରକାର ଦର୍ଶାଇଛନ୍ତି ଯେ ଆରଏସଏସ କେଉଁମାନଙ୍କ ବିରୁଦ୍ଧରେ କୌଣସି ନାଗରିକ ଅବମାନନାକୁ ସମର୍ଥନ କରୁନାହାନ୍ତି?
Reference: {'text': ['ବ୍ରିଟିଶ ସରକାର'], 'answer_start': [718]}


In [34]:
model.save_pretrained('./models')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
